# Exploration

A notebook used to explore FRED data

## Setup    

In [ ]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Setup autoreload of modules
%load_ext autoreload
%autoreload 2


In [ ]:
from update_path import ensure_src_in_path

ensure_src_in_path()

In [ ]:
# Get the FRED API client

from fred_data import api

%autoreload 2

fred_client = api.FredApiClient()

## Data Exploration

In [ ]:
# Fetch all releases
from fred_data.releases import get_releases

releases = get_releases(fred_client)
releases_df = releases.releases
releases_df

In [ ]:
# Get an idea of the amount/types of data stored in releases
print(f"Release summary")
print(f"Count: {releases.count}")
print(f"Realtime range: {releases.realtime_start} to {releases.realtime_end}, Duration: {releases.realtime_end - releases.realtime_start}")
releases.releases.describe()